<a href="https://colab.research.google.com/github/DonguYang/Data-Analysis/blob/circleci-project-setup/%EC%A0%9C%EC%A1%B0%EC%97%85%EB%8F%84%EB%A9%94%EC%9D%B8_%EC%8B%AC%ED%99%94%ED%8E%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **① 문제상황**
---
> **시나리오** 🏭

```
'반도체 공정 이상 탐지' 프로젝트 팀은 반도체 제조의 복잡한 프로세스에 대한 새로운 프로젝트를 맡게 되었습니다. 이 프로젝트는 센서와 설비에서
수집된 다양한 신호와 변수들을 세심하게 모니터링하는 일로부터 시작됩니다. 이 신호들이 모두 동일한 중요성을 가지는 것은 아니라는 점이 현업이
요구한 내용의 핵심이며 우리 팀이 풀어가야하는 핵심문제입니다.

우리가 분석할 데이터는 유용한 정보, 관련 없는 정보, 그리고 잡음이 혼합된 복잡합니다. 우리에게는 실제로 필요한 것보다 많은 신호들이 주어져 있으며,
일반적으로 엔지니어들은 각각의 신호 유형을 하나의 독특한 특성으로 보고 접근합니다. Feature Engineering(특성 선택 기법)을 적용하여,
우리는 가장 관련성 높은 신호들을 선별해 내며, 이러한 신호들을 활용해 공정에서 발생할 수 있는 수율 이탈의 주요 원인들을 파악하려 합니다.

이 과정은 공정의 효율성을 향상시키고, 생산 비용을 절감할 수 있는 다양한 인사이트를 제공할 수 있을 것이라 기대됩니다. 이 신호들은 또한
반도체의 생산의 Pass/Fail 을 예측하는 데 사용될 수 있는 귀중한 특성으로 활용됩니다. 다양한 특성 조합을 탐색하고 실험함으로써,
우리는 수율 유형에 중대한 영향을 미치는 핵심 신호들을 식별할 수 있습니다.

정리하자면 우리 팀의 목표는 바로 반도체 공정 프로세스의 합격/불합격 수율을 예측할 수 있는 가장 의미 있는 신호를 찾아내고,
반도체 제조 과정의 품질과 효율성을 한 단계 끌어올리는 것입니다.

```

In [1]:
# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

# ▶ Google drive mount or 폴더 클릭 후 구글드라이브 연결
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split

#모델 불러오기
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
#평가 모듈 불러오기
from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv('/content/drive/MyDrive/uci-secom.csv')

In [35]:
df = df.drop('Time',axis=1)

# EDA


In [5]:
df.shape

(1567, 592)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [22]:
df.isna().sum().sort_values(ascending=False)

158          1429
292          1429
293          1429
157          1429
85           1341
             ... 
386             0
361             0
360             0
359             0
Pass/Fail       0
Length: 592, dtype: int64

In [12]:
# 결측치의 비율이 10%(약 150개)를 넘어가는 특성의 갯수
len(df.isna().sum().sort_values(ascending=False)[df.isna().sum() > 150])

52

In [26]:
df['Pass/Fail'].value_counts() #0.066

-1    1463
 1     104
Name: Pass/Fail, dtype: int64

# 1. 결측치

- 결측치가 많은 센서들이 눈에 띈다. 90% 정도가 결측치인 특성들도 있으며, 10% 이상의 비율을 가지는 센서가 52개로 전체 센서 중 9%에 달한다.
- 또한 결측치의 갯수가 같은 변수들이 있다. 이 변수들의 na가 아닌 값들을 살펴봤을 때, 같은 척도의 수치로 보이지 않아 같은 경우 **센서가 반응하지만 다른 것을 측정하는 경우이지 않을까?** 라고 생각했다.
- 결측치가 적은 경우, 평균값을 사용할 수 있겠지만 위와 같은 경우 어렵다고 판단, 결측치를 살펴볼 필요가 있다.

In [13]:
high_na_cols = df.isna().sum().sort_values(ascending=False)[df.isna().sum() > 150].index



In [46]:
pass_fail_counts = pd.DataFrame(index=df.columns, columns=['Pass', 'Fail'])

for col in df.columns:
    na_rows = df[df[col].isna()]
    pass_fail_counts.loc[col, 'Pass'] = na_rows[na_rows['Pass/Fail'] == 1].shape[0]
    pass_fail_counts.loc[col, 'Fail'] = na_rows[na_rows['Pass/Fail'] == -1].shape[0]


pass_fail_counts['pass_ratio'] = pass_fail_counts['Pass'] / (pass_fail_counts['Pass'] + pass_fail_counts['Fail']).replace(0, 1)

pass_fail_counts.sort_values('pass_ratio',ascending=False).drop_duplicates()

,Pass,Fail,pass_ratio
295,1,1,0.5
467,1,3,0.25
410,1,6,0.142857
19,1,9,0.1
563,24,249,0.087912
244,74,944,0.072692
492,93,1248,0.069351
546,18,242,0.069231
292,96,1333,0.06718
580,59,890,0.062171


In [45]:
pass_fail_counts_no_na = pd.DataFrame(index=df.columns, columns=['Pass', 'Fail'])

for col in df.columns:
    non_na_rows = df[df[col].notna()]
    pass_fail_counts_no_na.loc[col, 'Pass'] = non_na_rows[non_na_rows['Pass/Fail'] == 1].shape[0]
    pass_fail_counts_no_na.loc[col, 'Fail'] = non_na_rows[non_na_rows['Pass/Fail'] == -1].shape[0]

pass_fail_counts_no_na['pass_ratio'] = pass_fail_counts_no_na['Pass'] / (pass_fail_counts_no_na['Pass'] + pass_fail_counts_no_na['Fail']).replace(0, 1)

pass_fail_counts_no_na.sort_values('pass_ratio', ascending=False).drop_duplicates()


,Pass,Fail,pass_ratio
345,68,705,0.087969
247,71,781,0.083333
578,45,573,0.072816
497,103,1413,0.067942
253,104,1439,0.067401
140,104,1449,0.066967
357,104,1451,0.066881
533,104,1454,0.066752
269,104,1455,0.066709
474,104,1456,0.066667


In [52]:
# 전체 비율보다 높은 비율을 보이는 경우 찾기

# 1) 결측치일 경우 양품 가능성이 높아지는 센서들
na_pass_up = pass_fail_counts[(pass_fail_counts['pass_ratio']>=0.067) & (pass_fail_counts['Fail']>200)].index

# 2) 결측치 아닐 경우 양품 가능성이 높아지는 센서들
not_na_pass_up = pass_fail_counts_no_na[(pass_fail_counts_no_na['pass_ratio']>=0.067) & (pass_fail_counts['Fail']>200)].index

In [53]:
print(len(na_pass_up),len(not_na_pass_up))

40 12


In [54]:
df[not_na_pass_up].isna().sum()

72     794
73     794
112    715
247    715
345    794
346    794
385    715
519    715
578    949
579    949
580    949
581    949
dtype: int64

In [55]:
df[na_pass_up].isna().sum()

85     1341
109    1018
110    1018
111    1018
157    1429
158    1429
220    1341
244    1018
245    1018
246    1018
292    1429
293    1429
358    1341
382    1018
383    1018
384    1018
492    1341
516    1018
517    1018
518    1018
546     260
547     260
548     260
549     260
550     260
551     260
552     260
553     260
554     260
555     260
556     260
557     260
562     273
563     273
564     273
565     273
566     273
567     273
568     273
569     273
dtype: int64

# 2. NA값 처리하기

- NA의 비율이 낮다면 앞서 말했듯 평균값으로 처리하는게 좋겠지만 비율이 높다면(절반 이상) 처리하기 어렵다. 이런 경우 NA가 아닌 값끼리 scaler을 적용하고 NA인 값은 0으로 처리하는 것을 생각했다.
- 이러한 처리를 하기 전/후의 모델의 정확성을 측정해보자.

In [64]:
# NA를 평균값 처리
target = df['Pass/Fail']
target = target.replace(-1,0)

features = df.drop('Pass/Fail',axis=1)
features_mean = features.fillna(features.mean())

In [66]:
# NA를 0으로 처리
features_0 = features.fillna(0)

In [61]:
# 사용할 모델
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42)
}


In [70]:
results_df = pd.DataFrame(index=models.keys(), columns=['mean_accuracy', 'zero_accuracy'])

for data_name, features_data in [('features_mean', features_mean), ('features_0', features_0)]:

    X_train, X_test, y_train, y_test = train_test_split(features_data, target, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    for model_name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        results_df.at[model_name, data_name] = accuracy

results_df = results_df[['features_mean', 'features_0']]

results_df

,features_mean,features_0
Random Forest,0.923567,0.920382
XGBoost,0.926752,0.920382
Gradient Boosting,0.907643,0.901274
Logistic Regression,0.888535,0.869427


In [76]:
def feature_importance_analysis(features_df, target_df, model_name):
    # 모델 선택
    selected_model = models[model_name]

    X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 모델 학습
    selected_model.fit(X_train_scaled, y_train)

    # Feature importance 추출
    if hasattr(selected_model, 'feature_importances_'):
        feature_importances = selected_model.feature_importances_
    elif hasattr(selected_model, 'coef_'):
        feature_importances = np.abs(selected_model.coef_[0])
    else:
        feature_importances = np.zeros(len(features_df.columns))

    # Top 30 Feature만 선택
    top30_features = features_df.columns[np.argsort(feature_importances)[-30:]]

    # Feature importance를 DataFrame으로 반환
    feature_importance_df = pd.DataFrame({'Feature': top30_features, 'Importance': feature_importances[np.argsort(feature_importances)[-30:]]})

    return feature_importance_df


In [80]:
models

{'Random Forest': RandomForestClassifier(random_state=42),
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=42, ...),
 'Gradient Boosting': GradientBoostingClassifier(random_state=42),
 'Logistic Regression': LogisticRegression(random_state=42)}

In [ ]:
combined_feature_importance_df = pd.DataFrame()

for model_name in models.keys():
    feature_importance_df = feature_importance_analysis(features_0, target, model_name)
    combined_feature_importance_df = pd.concat([combined_feature_importance_df, feature_importance_df], axis=1)




In [81]:
combined_feature_importance_df.columns = ['RF_feature','RF_importance','XGB_feature','XGB_importance','GB_feature','GB_importance','LR_feature','LR_importance']


,RF_feature,RF_importance,XGB_feature,XGB_importance,GB_feature,GB_importance,LR_feature,LR_importance
0,359,0.004474,64,0.008417,525,0.009575,33,0.623976
1,435,0.004499,156,0.008617,527,0.009643,71,0.624473
2,331,0.004522,10,0.008678,585,0.010448,86,0.637859
3,519,0.004545,168,0.009252,301,0.010573,305,0.644659
4,426,0.004581,471,0.009334,51,0.010970,188,0.649478
5,88,0.004689,155,0.009481,218,0.011007,113,0.662713
6,27,0.004722,301,0.009490,114,0.011015,332,0.671271
7,26,0.004744,468,0.009945,104,0.011173,377,0.679559
8,348,0.004809,333,0.010242,2,0.011222,150,0.696878
9,583,0.004809,267,0.010650,121,0.011862,471,0.700561


In [83]:
# combined_feature_importance_df에서 feature 열들을 추출
features = combined_feature_importance_df.iloc[:, 0::2].values.flatten()

# 중복된 feature 찾기
features_list = features.tolist()
duplicated_features = set(x for x in features_list if features_list.count(x) > 1)

# 중복된 feature 및 그 중복 횟수 출력
for feature in duplicated_features:
    count = features_list.count(feature)
    print(f"Feature: {feature}, Duplicated Count: {count}")


Feature: 301, Duplicated Count: 2
Feature: 155, Duplicated Count: 2
Feature: 25, Duplicated Count: 2
Feature: 59, Duplicated Count: 4
Feature: 488, Duplicated Count: 2
Feature: 129, Duplicated Count: 2
Feature: 16, Duplicated Count: 2
Feature: 71, Duplicated Count: 2
Feature: 341, Duplicated Count: 2
Feature: 441, Duplicated Count: 2
Feature: 51, Duplicated Count: 2
Feature: 40, Duplicated Count: 2
Feature: 64, Duplicated Count: 4
Feature: 33, Duplicated Count: 2
Feature: 104, Duplicated Count: 2
Feature: 471, Duplicated Count: 2
Feature: 333, Duplicated Count: 3


- 4개의 모델에서 중복되었던 변수들은 다음과 같다.
- 결측치가 없는 변수들이 대체적으로 중요 변수로 꼽혔다고 볼 수 있다.

In [85]:
key_censor = ['301','155','25','59','488','129','16','71','341','441','51','40','64','33','104','471','333']
df[key_censor].isna().sum()

301     2
155    10
25      2
59      7
488    24
129     9
16      3
71      6
341     6
441     1
51      1
40     24
64      7
33      1
104     2
471     6
333     6
dtype: int64

In [100]:
censor_xgb = combined_feature_importance_df['XGB_feature']

df[censor_xgb].isna().sum()

64       7
156      0
10       2
168      2
471      6
155     10
301      2
468      7
333      6
267      8
30       2
72     794
83       1
499      2
412     14
47       1
119      0
4       14
334      6
138     14
485     24
59       7
270      5
60       6
474      7
170      1
363     51
250      0
275     14
17       3
dtype: int64

In [103]:
df.columns[df.isna().sum() == 794]

Index(['72', '73', '345', '346'], dtype='object')

# 결론

-  가장 좋았던 XGB모델의 top 30 변수들의 결측치를 확인해보니 몇 가지 특징이 있었다.


1.   결측치가 많은(250개 이상, 전체의 15%이상이 결측치)인 변수가 13개
2.   794의 경우 예외적으로 많은 결측치를 보유.



이번 주 시간이 없어서 결론과 문제 정의 부분을 제대로 정의하지 못했습니다.

관련하여 질문들이 있어 정리하여 따로 질문 드리도록 하겠습니다..ㅠㅠ